<a href="https://colab.research.google.com/github/souravmsi/Text-Generation/blob/main/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://www.gutenberg.org/files/1661/1661-0.txt -O book.txt

--2021-07-16 15:33:13--  https://www.gutenberg.org/files/1661/1661-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607430 (593K) [text/plain]
Saving to: ‘book.txt’

book.txt            100%[===================>] 593.19K  --.-KB/s    in 0.06s   

2021-07-16 15:33:13 (10.0 MB/s) - ‘book.txt’ saved [607430/607430]



In [3]:
text = open('book.txt', 'r', encoding = 'utf-8').read()
text = text.lower()

In [5]:
sentences = text.split('\n')

In [6]:
import tensorflow as tf
import keras
from keras_preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [7]:
tokenizer = Tokenizer(oov_token='<UNK>')

In [8]:
tokenizer.fit_on_texts(sentences)

In [9]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

8915

In [10]:
sequences = tokenizer.texts_to_sequences(sentences)

In [11]:
input_sequences = []
for sequence in sequences:
  for i in range(1, len(sequence)):
    n_gram_sequence = sequence[:i+1]
    input_sequences.append(n_gram_sequence)

In [12]:
print(input_sequences[0:2])

[[4775, 145], [4775, 145, 132]]


In [13]:
print(sequences[0:2])

[[4775, 145, 132, 886, 5, 2, 1093, 5, 130, 35, 46, 656, 2661, 2662], []]


In [14]:
max_seq_len = max([len(seq) for seq in input_sequences])
max_seq_len

20

In [15]:
padded_sequences = pad_sequences(input_sequences, max_seq_len)

In [16]:
print(padded_sequences[0:2])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0 4775  145]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 4775  145  132]]


In [17]:
import numpy as np
padded_sequences = np.array(padded_sequences)

In [18]:
x = padded_sequences[:,:-1]
labels = padded_sequences[:,-1]

In [19]:
y = tf.keras.utils.to_categorical(labels, num_classes=vocab_size)

In [20]:
y.shape

(101575, 8915)

In [21]:
x.shape

(101575, 19)

In [22]:
 from tensorflow.keras.models import Sequential
 from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
 from tensorflow.keras.optimizers import Adam

In [23]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_seq_len-1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer = adam, metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 100)           891500    
_________________________________________________________________
bidirectional (Bidirectional (None, 512)               731136    
_________________________________________________________________
dense (Dense)                (None, 8915)              4573395   
Total params: 6,196,031
Trainable params: 6,196,031
Non-trainable params: 0
_________________________________________________________________


In [24]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_acc', min_delta=0.01)

In [25]:
model.fit(x, y, epochs=50, verbose=1, batch_size=512, callbacks=[es])

Epoch 1/50
199/199 [==============================] - 17s 43ms/step - loss: 6.1958 - acc: 0.0855
Epoch 2/50
199/199 [==============================] - 8s 43ms/step - loss: 5.3034 - acc: 0.1432
Epoch 3/50
199/199 [==============================] - 8s 42ms/step - loss: 4.7675 - acc: 0.1726
Epoch 4/50
199/199 [==============================] - 8s 43ms/step - loss: 4.2551 - acc: 0.2008
Epoch 5/50
199/199 [==============================] - 9s 43ms/step - loss: 3.7589 - acc: 0.2434
Epoch 6/50
199/199 [==============================] - 9s 43ms/step - loss: 3.3119 - acc: 0.2960
Epoch 7/50
199/199 [==============================] - 9s 43ms/step - loss: 2.9312 - acc: 0.3514
Epoch 8/50
199/199 [==============================] - 9s 43ms/step - loss: 2.6254 - acc: 0.4026
Epoch 9/50
199/199 [==============================] - 8s 43ms/step - loss: 2.3750 - acc: 0.4468
Epoch 10/50
199/199 [==============================] - 9s 43ms/step - loss: 2.1634 - acc: 0.4847
Epoch 11/50
199/199 [=================

In [40]:
#Predicting Part
seed_text = "I could not help laughing at the ease with which he explained his process of deduction"
next_words = 100
for _ in range(next_words):
  sequence = tokenizer.texts_to_sequences([seed_text])
  padded = pad_sequences(sequence, maxlen=max_seq_len-1)
  predicted = model.predict_classes(padded, verbose=0)
  output_word = ''
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text = seed_text + " " + output_word
print(seed_text)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


I could not help laughing at the ease with which he explained his process of deduction that is for i have been obliged to work out the house in the morning when the house had been so that he is the precious stone the stone is a bad face been for a most day house to if you wish to it it is it ’ i said when i had been been my had lost so so i by my mind that my boy was out to do it to be ’ said he and i can to do so so to say ’ i said when the police it is a by a large “g”
